In [3]:
print("OK")

OK


In [4]:
%pwd

'c:\\Users\\educa\\Desktop\\medical_assistant-chat_bot\\research'

In [5]:
import os 
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\educa\\Desktop\\medical_assistant-chat_bot'

In [7]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import transformers
# import sentence_transformers



c:\Users\educa\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [9]:
extracted_data = load_pdf("data/")

In [12]:
#extracted_data

In [13]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [15]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [16]:
embeddings = download_hugging_face_embeddings()

C:\Users\educa\AppData\Local\Temp\ipykernel_8600\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
# query_result

In [19]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [27]:
from pinecone import Pinecone
from pinecone import Pinecone, ServerlessSpec
from pinecone.grpc import PineconeGRPC as Pinecone

pc = Pinecone(api_key= PINECONE_API_KEY)

index_name = "medicalbot"
pc.create_index(
    name = index_name,
    dimension =384,
    metric= "cosine",
    spec = ServerlessSpec(
        cloud ="aws",
        region ="us-east-1"
    )
)



{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-qcj6l0e.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [28]:
import os
os.environ["PINECONE_API_KEY"]= PINECONE_API_KEY

In [30]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name= index_name,
    embedding=embeddings,

)

In [31]:
#load existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings

)


In [32]:
docsearch

In [33]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [34]:
retrieved_docs= retriver.invoke("what is Acne")